In [1]:
import pandas as pd
import os
import bs4
import pickle
import requests
from bs4 import BeautifulSoup
import sys
import matplotlib.pyplot as plt

In [125]:
company_list = ['Apple']
years = ['2019', '2018']

filing_type = '10-K'

In [126]:
key_path = os.path.join('..','Financials', 'Ticker_and_CIK', 'Key.csv')

key_df = pd.read_csv(key_path)

key_dict = {}

for query in range(len(company_list)):
    try:
        company_df = key_df[key_df['Company'].str.contains(company_list[query].title())]
        key_dict[company_df.iloc[0, 1]] = [company_df.iloc[0, 0], company_df.iloc[0, 2]]
    except IndexError:
        print(f"{company_list[query]} could not be found")

if key_dict == {}:
    print ("Whoops, none of your companies were on the S&P 500. Please try again!")
else:
    print (f"Success! We were able to find {len(key_dict.keys())} of your queries in the S&P 500")
    print (f"We found: {list(key_dict.keys())} in the S&P 500")
    print (key_dict)

Success! We were able to find 1 of your queries in the S&P 500
We found: ['Apple Inc.'] in the S&P 500
{'Apple Inc.': ['AAPL', 320193]}


In [119]:
#Unmodified Base Code found at https://www.codeproject.com/Articles/1227765/Parsing-XBRL-with-Python
#I added the ability for the code to query through a list of companies for multiple years and multiple quarters
#and save the statements 

published_date_dict = {}
as_of_date_dict = {}

for companies_found in range(len(key_dict.keys())):
    
    company_name = list(key_dict.keys())[companies_found]
    ticker = key_dict.get(company_name)[0]
    
    cik = key_dict.get(company_name)[1]
    dateb = f'{years[0]}-12-31'
    
    # Obtain HTML for search page
    base_url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type={}&dateb={}"
    edgar_resp = requests.get(base_url.format(cik, filing_type, dateb))
    edgar_str = edgar_resp.text
    
    doc_links = []
    published_dates = []
    as_of_dates = []
            
    # Find the document link
    soup = BeautifulSoup(edgar_str, 'html.parser')
    table_tag = soup.find('table', class_='tableFile2')
    rows = table_tag.find_all('tr')
        
    for year in years:
        for row in rows:
            cells = row.find_all('td')
            if len(cells) > 3:
                if year in cells[3].text:
                    doc_link = 'https://www.sec.gov' + cells[1].a['href']
                    published_date = cells[3].text
                    published_dates.append(published_date)
                    
                    # Obtain HTML for document page
                    doc_resp = requests.get(doc_link)
                    doc_str = doc_resp.text
                    # Find the XBRL link
                    xbrl_link = ''
                    soup_doc = BeautifulSoup(doc_str, 'html.parser')
                    table_tag_doc = soup_doc.find('table', class_='tableFile', summary='Data Files')
                    rows_doc = table_tag_doc.find_all('tr')
                    for row_doc in rows_doc:
                        cells_doc = row_doc.find_all('td')
                        if len(cells_doc) > 3:
                            if int(year) > 2018:
                                if 'XML' in cells_doc[3].text:
                                    xbrl_link = 'https://www.sec.gov' + cells_doc[2].a['href']
                            else:
                                if 'INS' in cells_doc[3].text:
                                    xbrl_link = 'https://www.sec.gov' + cells_doc[2].a['href']
                    # Obtain XBRL text from document
                    xbrl_resp = requests.get(xbrl_link)
                    xbrl_str = xbrl_resp.text
               
                    # Find and print stockholder's equity
                    soup = BeautifulSoup(xbrl_str, 'lxml')
                    tag_list = soup.find_all()
                    
                    df = pd.DataFrame()
                    
                    for tag in tag_list:
                        if 'schemaref' in tag.name:
                            ignored_href_index = len(ticker) + 1
                            as_of_date = tag['xlink:href'][ignored_href_index:(ignored_href_index+8)]
                            as_of_dates.append(as_of_date)
                            
                         #Modified from the original; The original searched through to find specific line items
                        #Modified to search through every 10-Q per given year, and extract any financial line item into its own 10-Q

                        if tag.name.find('us-gaap:') != -1:
                            try: 
                                if float(tag.text) > 0:
                                    i = [f'{tag.name}', tag.text, tag['contextref']]
                                    i_series = pd.Series(i)
                                    df = df.append(i_series, ignore_index = True)
                            except ValueError:
                                pass

                    folder_path = os.path.join('..','Financials',f'{filing_type}s',f'{ticker}')
                    file_path = os.path.join('..','Financials',f'{filing_type}s',f'{ticker}', f'{ticker}_{published_date}.csv')

                    try:
                        os.mkdir(folder_path)
                    except FileExistsError:
                        pass
                    df.to_csv(file_path, index = False)

    published_date_dict[ticker] = published_dates
    as_of_date_dict[ticker] = as_of_dates
    
print(as_of_date_dict)
print(published_date_dict)

{'AAPL': ['20180630', '20180331', '20171230', '20170701', '20170401', '20161231'], 'MMM': ['20180930', '20180630', '20180331', '20170930', '20170630', '20170331']}
{'AAPL': ['2018-08-01', '2018-05-02', '2018-02-02', '2017-08-02', '2017-05-03', '2017-02-01'], 'MMM': ['2018-10-25', '2018-07-26', '2018-05-08', '2017-10-31', '2017-08-01', '2017-05-03']}


In [60]:
date = "2017-02-01"
year = 2017

#Get the saved 10-K     
file_path = os.path.join('..','Financials',f'{filing_type}s',f'{ticker}', f'{ticker}_{date}.csv')
df = pd.read_csv(file_path)

#Find the Current Ratio
c_assets_df = df[df['0'].str.contains('us-gaap:assetscurrent')]
print(c_assets_df)

print(c_assets_df['2'][c_assets_df['2']==2017])

reduced_df = c_assets_df[c_assets_df['2'].str.contains(f'{year}')]

#if len(reduced_df ==)
len(reduced_df)

                        0             1         2
25  us-gaap:assetscurrent  1.068690e+11  FI2016Q4
26  us-gaap:assetscurrent  1.033320e+11  FI2017Q1
Series([], Name: 2, dtype: object)


1

In [81]:
test_date = '20170201'

y = test_date [0:4]
m = test_date [4:6]
d = test_date [6:8]
year_check = int(y) + 1

date_formats = [f'{y}{m}{d}', f'{y}-{m}-{d}', f'{m.lstrip("0")}_{d.lstrip("0")}_{y}']


quarter_list = ['Q4', 'Q3', "Q2", "Q1"]

context_ref = c_assets_df['2']



test_df = pd.DataFrame()


if len(context_ref) == 1:
    test_df = test_df.append(c_assets_df)
elif len(context_ref) == 0:
    print('Error, There are no Entries for that')
else:
    while year_check >= (int(y)-1):
        year_check_df = c_assets_df[context_ref.str.contains(f'{year_check}')]
        
        if len(year_check_df) == 1:
            test_df = test_df.append(year_check_df)
            break
        else: 
            for item in date_formats:
                if len(c_assets_df[context_ref.str.contains(f'{item}')]) == 1:
                    test_df = test_df.append(c_assets_df[context_ref.str.contains(f'{item}')])
                    
                    break
            else:
                for quarter in quarter_list:
                    if len(c_assets_df[context_ref.str.contains(f'{quarter}{year_check}')]) == 1:
                        test_df = test_df.append(c_assets_df[context_ref.str.contains(f'{quarter}{year_check}')])
                        
                        break
                    elif len(c_assets_df[context_ref.str.contains(f'{year_check}{quarter}')]) == 1:
                        test_df = test_df.append(c_assets_df[context_ref.str.contains(f'{year_check}{quarter}')])
                        break
        year_check = year_check - 1
print(test_df)

                        0             1         2
26  us-gaap:assetscurrent  1.033320e+11  FI2017Q1


In [ ]:
curr_ratio_dict = {}
per_change_dict = {}
bvps_dict = {}
eps_dict = {}
week_change_dict = {}
month_change_dict = {}
stock_price_dict = {}

for companies_found in range(len(published_date_dict.keys())):
    
    ticker = list(published_date_dict.keys())[companies_found]
    
    comp_curr_ratio_dict = {}
    comp_per_change_dict = {}
    comp_bvps_dict = {}
    comp_eps_dict = {}
    comp_week_change_dict = {}
    comp_month_change_dict = {}
    comp_stock_price_dict = {}
    
    for date in published_date_dict[ticker]: 
        
        y = date [0:4]
        m = date [5:7]
        d = date [9:10]
        
        stock_date = f'{m}/{d}/{y}'
        
        stock_path = os.path.join('..','Stock_Data','API_Stock_Data',f'{ticker}', f'{ticker}_{y}.csv')
        stock_data_df = pd.read_csv(stock_path)
        
        
        def find_entry():
            for r in c_assets_df.iloc[:, 2]:
                #Need to use date format finder instead
                cref_list.append(int(r[-10:-3]))
        
        #Get the saved 10-K     
        file_path = os.path.join('..','Financials',f'{filing_type}s',f'{ticker}', f'{ticker}_{date}.csv')
        df = pd.read_csv(file_path)
        
        #Find the Current Ratio
        c_assets_df = df[df['0'].str.contains('us-gaap:assetscurrent')]
        cref_list = []
        find_entry()
        curr_assets = c_assets_df[c_assets_df['2'].str.contains(str(max(cref_list)))].iloc[0,1]
        
        c_liab_df = df[df['0'].str.contains('us-gaap:liabilitiescurrent')]
        cref_list = []
        find_entry()
        curr_liab = c_liab_df[c_liab_df['2'].str.contains(str(max(cref_list)))].iloc[0,1]
        
        curr_ratio = (curr_assets / curr_liab).round(2)
        comp_curr_ratio_dict[date] = curr_ratio
        
        #Find Book-Value Per Share
        share_equity_df = df[df['0'].str.contains('us-gaap:stockholdersequity')]
        cref_list = []
        find_entry()
        share_equity = share_equity_df[share_equity_df['2'].str.contains(str(max(cref_list)))].iloc[0,1]
        
        shares_out_df = df[df['0'].str.contains('us-gaap:commonstocksharesoutstanding')]
        cref_list = []
        find_entry()
        shares_out = shares_out_df[shares_out_df['2'].str.contains(str(max(cref_list)))].iloc[0,1]
        
        bvps = share_equity / shares_out
        comp_bvps_dict[date] = bvps
        
        
        #Find Earnings per Share
        inc_df = df[df['0'].str.contains('us-gaap:comprehensiveincomenetoftax')]
        cref_list = []
        find_entry()
        inc = inc_df[inc_df['2'].str.contains(str(max(cref_list)))].iloc[0,1]
        eps = inc / shares_out
        comp_eps_dict[date] = eps.round(2)
        
        #Find the pertinent Stock Data
        release_day_data = stock_data_df[stock_data_df['Time'] == date]
        release_day_close = release_day_data['Last']
        
        release_day_index = int(release_day_data.index.values)
        
        prior_day_data = stock_data_df.iloc[release_day_index-1]
        next_day_data = stock_data_df.iloc[release_day_index+1]
        five_day_data = stock_data_df.iloc[release_day_index+4]
        month_day_data = stock_data_df.iloc[release_day_index+19]
        
        prior_day_close = prior_day_data['Last']
        next_day_close = next_day_data['Last']
        five_day_close = five_day_data['Last']
        month_day_close = month_day_data['Last']
        
        comp_stock_price_dict[date] = int(next_day_close)
        
        per_change = ((next_day_close / prior_day_close) - 1) * 100
        comp_per_change_dict[date] = per_change.round(2)
        
        week_change = ((five_day_close / prior_day_close) - 1) * 100
        comp_week_change_dict[date] = week_change.round(2)
        
        month_change = ((month_day_close / prior_day_close) -1) * 100
        comp_month_change_dict[date] = month_change.round(2)
        
    curr_ratio_dict[ticker] = comp_curr_ratio_dict
    per_change_dict[ticker] = comp_per_change_dict
    bvps_dict[ticker] = comp_bvps_dict
    eps_dict[ticker] = comp_eps_dict
    week_change_dict[ticker] = comp_week_change_dict
    month_change_dict[ticker] = comp_month_change_dict
    stock_price_dict[ticker] = comp_stock_price_dict
print("Success")

In [120]:
def find_entry():
    y = test_date [0:4]
    m = test_date [4:6]
    d = test_date [6:8]
    year_check = int(y) + 1

    date_formats = [f'{y}{m}{d}', f'{y}-{m}-{d}', f'{m.lstrip("0")}_{d.lstrip("0")}_{y}']

    quarter_list = ['Q4', 'Q3', "Q2", "Q1"]
    context_ref = c_assets_df['2']


    test_df = pd.DataFrame()


    if len(context_ref) == 1:
        test_df = test_df.append(c_assets_df)
    elif len(context_ref) == 0:
        print('Error, There are no Entries for that')
    else:
        while year_check >= (int(y)-1):
            year_check_df = c_assets_df[context_ref.str.contains(f'{year_check}')]

            if len(year_check_df) == 1:
                test_df = test_df.append(year_check_df)
                break
            else: 
                for item in date_formats:
                    if len(c_assets_df[context_ref.str.contains(f'{item}')]) == 1:
                        test_df = test_df.append(c_assets_df[context_ref.str.contains(f'{item}')])

                        break
                else:
                    for quarter in quarter_list:
                        if len(c_assets_df[context_ref.str.contains(f'{quarter}{year_check}')]) == 1:
                            test_df = test_df.append(c_assets_df[context_ref.str.contains(f'{quarter}{year_check}')])

                            break
                        elif len(c_assets_df[context_ref.str.contains(f'{year_check}{quarter}')]) == 1:
                            test_df = test_df.append(c_assets_df[context_ref.str.contains(f'{year_check}{quarter}')])
                            break
            year_check = year_check - 1
    print(test_df)

    
for companies_found in range(len(published_date_dict.keys())):
    ticker = list(published_date_dict.keys())[companies_found]
    
    comp_curr_ratio_dict = {}
    comp_per_change_dict = {}
    comp_bvps_dict = {}
    comp_eps_dict = {}
    comp_week_change_dict = {}
    comp_month_change_dict = {}
    comp_stock_price_dict = {}
    
    print(ticker)
    
    for date_index in range(len(as_of_date_dict[ticker])):
        
        #Get the saved 10-K     
        file_path = os.path.join('..','Financials',f'{filing_type}s',f'{ticker}', f'{ticker}_{published_date_dict[ticker][date_index]}.csv')
        df = pd.read_csv(file_path)
        
        #Find the Current Ratio
        test_date = as_of_date_dict[ticker][date_index]
        print(test_date)
        
        c_assets_df = df[df['0'].str.contains('us-gaap:assetscurrent')]
        find_entry()
        
        c_assets_df = df[df['0'].str.contains('us-gaap:liabilitiescurrent')]
        find_entry()

AAPL
20180630
                        0             1         2
29  us-gaap:assetscurrent  1.157610e+11  FI2018Q3
                              0             1         2
457  us-gaap:liabilitiescurrent  8.854800e+10  FI2018Q3
20180331
                        0             1         2
37  us-gaap:assetscurrent  1.300530e+11  FI2018Q2
                              0             1         2
468  us-gaap:liabilitiescurrent  8.932000e+10  FI2018Q2
20171230
                        0             1         2
28  us-gaap:assetscurrent  1.438100e+11  FI2018Q1
                              0             1         2
419  us-gaap:liabilitiescurrent  1.157880e+11  FI2018Q1
20170701
                        0             1         2
35  us-gaap:assetscurrent  1.128750e+11  FI2017Q3
                              0             1         2
481  us-gaap:liabilitiescurrent  8.130200e+10  FI2017Q3
20170401
                        0             1         2
34  us-gaap:assetscurrent  1.019900e+11  FI2017Q2
  

In [140]:
def fetch_document_links(cells):
    doc_link = 'https://www.sec.gov' + cells[1].a['href']

    # Obtain HTML for document page
    doc_resp = requests.get(doc_link)
    doc_str = doc_resp.text
    # Find the XBRL link
    xbrl_link = ''
    soup_doc = BeautifulSoup(doc_str, 'html.parser')
    table_tag_doc = soup_doc.find('table', class_='tableFile', summary='Data Files')
    rows_doc = table_tag_doc.find_all('tr')
    for row_doc in rows_doc:
        cells_doc = row_doc.find_all('td')
        if len(cells_doc) > 3:
            if int(year) > 2018:
                if 'XML' in cells_doc[3].text:
                    return 'https://www.sec.gov' + cells_doc[2].a['href']
            else:
                if 'INS' in cells_doc[3].text:
                    return 'https://www.sec.gov' + cells_doc[2].a['href']

def fetch_financial_data(tag_list, as_of_dates, df_dict):
    
    row_num = 0
    
    for tag in tag_list:
        if 'schemaref' in tag.name:
            ignored_href_index = len(ticker) + 1
            as_of_date = tag['xlink:href'][ignored_href_index:(ignored_href_index+8)]
            as_of_dates.append(as_of_date)
            
        if tag.name.find('us-gaap:') != -1:
            try: 
                if float(tag.text) > 0:
                    i = [f'{tag.name}', tag.text, tag['contextref']]
                    df_dict[f'{row_num}'] = i
                    row_num = row_num + 1
            except ValueError:
                pass

#Unmodified Base Code found at https://www.codeproject.com/Articles/1227765/Parsing-XBRL-with-Python
#I added the ability for the code to query through a list of companies for multiple years and multiple quarters
#and save the statements 

published_date_dict = {}
as_of_date_dict = {}

for companies_found in range(len(key_dict.keys())):
    
    company_name = list(key_dict.keys())[companies_found]
    ticker = key_dict.get(company_name)[0]
    
    cik = key_dict.get(company_name)[1]
    dateb = f'{years[0]}-12-31'
    
    # Obtain HTML for search page
    base_url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type={}&dateb={}"
    edgar_resp = requests.get(base_url.format(cik, filing_type, dateb))
    edgar_str = edgar_resp.text
    
    doc_links = []
    published_dates = []
    as_of_dates = []
            
    # Find the document link
    soup = BeautifulSoup(edgar_str, 'html.parser')
    table_tag = soup.find('table', class_='tableFile2')
    rows = table_tag.find_all('tr')
        
    for year in years:
        for row in rows:
            cells = row.find_all('td')
            if len(cells) > 3:
                if year in cells[3].text:
                    published_date = cells[3].text
                    published_dates.append(published_date)

                    xbrl_link = fetch_document_links(cells)
                    # Obtain XBRL text from document
                    xbrl_resp = requests.get(xbrl_link)
                    xbrl_str = xbrl_resp.text
               
                    # Find and print stockholder's equity
                    soup = BeautifulSoup(xbrl_str, 'lxml')
                    tag_list = soup.find_all()
                    
                    df_dict = {}
                    
                    fetch_financial_data(tag_list, as_of_dates, df_dict)
                    df = pd.DataFrame.from_dict(df_dict, orient='index')
                    
                    folder_path = os.path.join('..','Financials',f'{filing_type}s',f'{ticker}')
                    file_path = os.path.join('..','Financials',f'{filing_type}s',f'{ticker}', f'{ticker}_{published_date}.csv')

                    try:
                        os.mkdir(folder_path)
                    except FileExistsError:
                        pass
                    df.to_csv(file_path, index = False)
                    print(file_path)

    published_date_dict[ticker] = published_dates
    as_of_date_dict[ticker] = as_of_dates
    
print(as_of_date_dict)
print(published_date_dict)

..\Financials\10-Ks\AAPL\AAPL_2019-10-31.csv
..\Financials\10-Ks\AAPL\AAPL_2018-11-05.csv
{'AAPL': ['20190928', '20180929']}
{'AAPL': ['2019-10-31', '2018-11-05']}
